In [24]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import string
punctuation = string.punctuation

conn = sqlite3.connect("zippedData/im.db")

A key measure of success for any movie is the amount of profit it brings in for its studio. None of our datasets include profit, so we will begin by determining how profitable each movie in our dataset is.

In [25]:
# need a column that denotes profit, so start by importing financial data
movie_finances = pd.read_csv('zippedData/tn.movie_budgets.csv')
movie_finances.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [26]:
# check data types of seres in movie_finances
movie_finances.dtypes

id                    int64
release_date         object
movie                object
production_budget    object
domestic_gross       object
worldwide_gross      object
dtype: object

In [27]:
# financial information is currently stored as a string, need to convert to integer
# start by stripping '$' from each string
movie_finances['production_budget'] = movie_finances['production_budget'].apply(lambda x: x.replace('$', ""))
movie_finances['domestic_gross'] = movie_finances['domestic_gross'].apply(lambda x: x.replace('$', ""))
movie_finances['worldwide_gross'] = movie_finances['worldwide_gross'].apply(lambda x: x.replace('$', ""))

In [28]:
# then strip ',' from each string
movie_finances['production_budget'] = movie_finances['production_budget'].apply(lambda x: x.replace(',', ""))
movie_finances['domestic_gross'] = movie_finances['domestic_gross'].apply(lambda x: x.replace(',', ""))
movie_finances['worldwide_gross'] = movie_finances['worldwide_gross'].apply(lambda x: x.replace(',', ""))

In [29]:
# finally, change dtype to int
movie_finances['production_budget'] = movie_finances['production_budget'].astype(int)
movie_finances['domestic_gross'] = movie_finances['domestic_gross'].astype(int)
movie_finances['worldwide_gross'] = movie_finances['worldwide_gross'].astype(int)

In [30]:
# confirm dtypes have been updated
movie_finances.dtypes

id                    int64
release_date         object
movie                object
production_budget     int64
domestic_gross        int64
worldwide_gross       int64
dtype: object

Now that we have cleaned the dataset, we can determine the domestic and worldwide profit for each movie. We will do so by subtracting the production budget from the domestic gross to determine domestic profit, and the from worldwide gross to determine worldwide profit.

In [47]:
# calculate domestic profit and save in a new column
movie_finances['domestic_profit'] = movie_finances['domestic_gross'] - movie_finances['production_budget']

# calculate worldwide profit and save in a new column
movie_finances['worldwide_profit'] = movie_finances['worldwide_gross'] - movie_finances['production_budget']

movie_finances

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,domestic_profit,worldwide_profit
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,335507625,2351345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,-169536125,635063875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,-307237650,-200237650
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,128405868,1072413963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,303181382,999721747
...,...,...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,0,-7000,-7000
5778,79,"Apr 2, 1999",Following,6000,48482,240495,42482,234495
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338,-3662,-3662
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0,-1400,-1400


### What movie rating should Microsoft aim for?
In addition to identifying ___ and ___, the rating a movie receives 

In [32]:
# get writer info from im.db writers table
writers = pd.read_sql("""
SELECT *
  FROM writers
  
""", conn)

In [33]:
# get director info from im.db directors table
directors = pd.read_sql("""
SELECT *
  FROM directors
""", conn)

In [34]:
# get information from im.db persons table
names = pd.read_sql('''
SELECT * 
FROM persons
''', conn)

In [35]:
# print column names to identify information to merge on
print(directors.columns)
print(names.columns)
print(writers.columns)

Index(['movie_id', 'person_id'], dtype='object')
Index(['person_id', 'primary_name', 'birth_year', 'death_year',
       'primary_profession'],
      dtype='object')
Index(['movie_id', 'person_id'], dtype='object')


In [36]:
# merge directors and names to obtain director names as "primary_name"
director_names = pd.merge(directors, names , how = 'inner', on = 'person_id')
director_names

,movie_id,person_id,primary_name,birth_year,death_year,primary_profession
0,tt0285252,nm0899854,Tony Vitale,1964.0,NaN,"producer,director,writer"
1,tt0462036,nm1940585,Bill Haley,NaN,NaN,"director,writer,producer"
2,tt0835418,nm0151540,Jay Chandrasekhar,1968.0,NaN,"director,actor,writer"
3,tt0835418,nm0151540,Jay Chandrasekhar,1968.0,NaN,"director,actor,writer"
4,tt0859635,nm0151540,Jay Chandrasekhar,1968.0,NaN,"director,actor,writer"
...,...,...,...,...,...,...
291166,tt8999892,nm10122247,C. Damon Adcock,NaN,NaN,None
291167,tt8999974,nm10122357,Daysi Burbano,NaN,NaN,"director,writer,cinematographer"
291168,tt9001390,nm6711477,Bernard Lessa,NaN,NaN,"director,writer,cinematographer"
291169,tt9001494,nm10123242,Tate Nova,NaN,NaN,"director,producer"


In [37]:
# merge writers and names to obtain writer names as "primary_name"
writer_names = pd.merge(writers, names , how = 'inner', on = 'person_id')
writer_names

,movie_id,person_id,primary_name,birth_year,death_year,primary_profession
0,tt0285252,nm0899854,Tony Vitale,1964.0,NaN,"producer,director,writer"
1,tt0438973,nm0175726,Steve Conrad,1968.0,NaN,"writer,producer,director"
2,tt2358925,nm0175726,Steve Conrad,1968.0,NaN,"writer,producer,director"
3,tt2543472,nm0175726,Steve Conrad,1968.0,NaN,"writer,producer,director"
4,tt0359950,nm0175726,Steve Conrad,1968.0,NaN,"writer,producer,director"
...,...,...,...,...,...,...
255866,tt8999892,nm10122247,C. Damon Adcock,NaN,NaN,None
255867,tt8999892,nm10122246,Bradley T. Castle,NaN,NaN,"actor,writer,producer"
255868,tt8999974,nm10122357,Daysi Burbano,NaN,NaN,"director,writer,cinematographer"
255869,tt9001390,nm6711477,Bernard Lessa,NaN,NaN,"director,writer,cinematographer"


In [38]:
# create a merged dataframe with writers and directors for each movie
directors_writers = pd.merge(director_names, writer_names, how = 'inner', on = 'movie_id').drop_duplicates().reset_index()

In [39]:
# drop extraneous columns
directors_writers.drop(['birth_year_x', 'death_year_x', 'primary_profession_x', 
                       'birth_year_y', 'death_year_y', 'primary_profession_y'],
                      axis = 1, inplace = True)

In [40]:
# rename columns to be clearer
directors_writers.rename(columns = {'person_id_x': 'director_id',
                                   'primary_name_x': 'director_name',
                                   'person_id_y': 'writer_id',
                                   'primary_name_y': 'writer_name'},
                        inplace = True)
directors_writers

,index,movie_id,director_id,director_name,writer_id,writer_name
0,0,tt0285252,nm0899854,Tony Vitale,nm0899854,Tony Vitale
1,1,tt0462036,nm1940585,Bill Haley,nm1940585,Bill Haley
2,2,tt0835418,nm0151540,Jay Chandrasekhar,nm0310087,Peter Gaulke
3,3,tt0835418,nm0151540,Jay Chandrasekhar,nm0841532,Gerry Swallow
4,6,tt0859635,nm0151540,Jay Chandrasekhar,nm0151540,Jay Chandrasekhar
...,...,...,...,...,...,...
254148,48154512,tt8998302,nm10121510,Daryl Boman,nm10121510,Daryl Boman
254149,48154513,tt8999892,nm10122247,C. Damon Adcock,nm10122247,C. Damon Adcock
254150,48154514,tt8999892,nm10122247,C. Damon Adcock,nm10122246,Bradley T. Castle
254151,48154517,tt8999974,nm10122357,Daysi Burbano,nm10122357,Daysi Burbano


In [41]:
# get information on movie ratings from file
movie_ratings = pd.read_table('zippedData/rt.movie_info.tsv', encoding = 'latin1')
movie_ratings

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,NaN,NaN,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",NaN,NaN,88 minutes,Paramount Vantage
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,NaN,"Jan 1, 1962","May 11, 2004",NaN,NaN,111 minutes,NaN
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",NaN,NaN,101 minutes,NaN


In [42]:
# common columns across both dataframes are director and writer
# will need to create a column that combines those names to merge the dataframes

directors_writers['director_writer'] = directors_writers['director_name'] + ", " + directors_writers['writer_name']
movie_ratings['director_writer'] = movie_ratings['director'] + ", " + movie_ratings['writer']

In [43]:
movie_ratings

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio,director_writer
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN,"William Friedkin, Ernest Tidyman"
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One,"David Cronenberg, David Cronenberg|Don DeLillo"
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN,"Allison Anders, Allison Anders"
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN,"Barry Levinson, Paul Attanasio|Michael Crichton"
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN,"Rodney Bennett, Giles Cooper"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,NaN,NaN,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema,NaN
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",NaN,NaN,88 minutes,Paramount Vantage,"Steve Barron, Terry Turner|Tom Davis|Dan Aykro..."
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,NaN,"Jan 1, 1962","May 11, 2004",NaN,NaN,111 minutes,NaN,NaN
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",NaN,NaN,101 minutes,NaN,"David Mickey Evans, David Mickey Evans|Robert ..."


In [44]:
# merge dataframes
movies = pd.merge(directors_writers, movie_ratings, how = 'inner', on = 'director_writer').drop_duplicates().reset_index()
movies

,level_0,index,movie_id,director_id,director_name,writer_id,writer_name,director_writer,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,0,290,tt1125929,nm0000431,Taylor Hackford,nm0414893,Mark Jacobson,"Taylor Hackford, Mark Jacobson",14,"""Love Ranch"" is a bittersweet love story that ...",R,Drama,Taylor Hackford,Mark Jacobson,"Jun 30, 2010","Nov 9, 2010",$,"134,904",117 minutes,NaN
1,1,33264,tt2693702,nm0000419,Jean-Luc Godard,nm0000419,Jean-Luc Godard,"Jean-Luc Godard, Jean-Luc Godard",691,Director Jean-Luc Godard tries to remain objec...,NR,Art House and International|Drama,Jean-Luc Godard,Jean-Luc Godard,"Sep 20, 1962","Aug 11, 1998",NaN,NaN,85 minutes,NaN
2,2,40132965,tt2400275,nm0000419,Jean-Luc Godard,nm0000419,Jean-Luc Godard,"Jean-Luc Godard, Jean-Luc Godard",691,Director Jean-Luc Godard tries to remain objec...,NR,Art House and International|Drama,Jean-Luc Godard,Jean-Luc Godard,"Sep 20, 1962","Aug 11, 1998",NaN,NaN,85 minutes,NaN
3,3,40132966,tt1438535,nm0000419,Jean-Luc Godard,nm0000419,Jean-Luc Godard,"Jean-Luc Godard, Jean-Luc Godard",691,Director Jean-Luc Godard tries to remain objec...,NR,Art House and International|Drama,Jean-Luc Godard,Jean-Luc Godard,"Sep 20, 1962","Aug 11, 1998",NaN,NaN,85 minutes,NaN
4,4,40133087,tt5749596,nm0000419,Jean-Luc Godard,nm0000419,Jean-Luc Godard,"Jean-Luc Godard, Jean-Luc Godard",691,Director Jean-Luc Godard tries to remain objec...,NR,Art House and International|Drama,Jean-Luc Godard,Jean-Luc Godard,"Sep 20, 1962","Aug 11, 1998",NaN,NaN,85 minutes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,48113936,tt5536610,nm0420982,Tamara Jenkins,nm0420982,Tamara Jenkins,"Tamara Jenkins, Tamara Jenkins",528,The last thing the two Savage siblings ever wa...,R,Comedy|Drama,Tamara Jenkins,Tamara Jenkins,"Nov 28, 2007","Apr 22, 2008",$,"6,426,953",114 minutes,Fox Searchlight Pictures
496,496,48122334,tt5873216,nm4354445,Anahita Ghazvinizadeh,nm4354445,Anahita Ghazvinizadeh,"Anahita Ghazvinizadeh, Anahita Ghazvinizadeh",1714,"Through the prism of J, notions of in-between-...",NR,Drama,Anahita Ghazvinizadeh,Anahita Ghazvinizadeh,NaN,NaN,NaN,NaN,80 minutes,NaN
497,497,48150588,tt0376479,nm0000191,Ewan McGregor,nm0738908,John Romano,"Ewan McGregor, John Romano",1236,"AMERICAN PASTORAL follows Seymour ""Swede"" Levo...",R,Drama,Ewan McGregor,John Romano,"Oct 21, 2016","Feb 7, 2017",$,"541,457",126 minutes,Lakeshore Entertainment
498,498,48151041,tt1720616,nm0922724,Jennifer Westfeldt,nm0922724,Jennifer Westfeldt,"Jennifer Westfeldt, Jennifer Westfeldt",304,Friends with Kids is a daring and poignant ens...,R,Comedy|Drama,Jennifer Westfeldt,Jennifer Westfeldt,"Mar 9, 2012","Jul 17, 2012",$,"5,600,000",107 minutes,Roadside Attractions


In [45]:
movies['rating'].value_counts()

R        205
NR       141
PG-13     78
PG        69
G          6
NC17       1
Name: rating, dtype: int64

In [46]:
basics = pd.read_sql("""
SELECT primary_title, runtime_minutes, genres
  FROM movie_basics
""", conn)

basics

,primary_title,runtime_minutes,genres
0,Sunghursh,175.0,"Action,Crime,Drama"
1,One Day Before the Rainy Season,114.0,"Biography,Drama"
2,The Other Side of the Wind,122.0,Drama
3,Sabse Bada Sukh,NaN,"Comedy,Drama"
4,The Wandering Soap Opera,80.0,"Comedy,Drama,Fantasy"
...,...,...,...
146139,Kuambil Lagi Hatiku,123.0,Drama
146140,Rodolpho Teóphilo - O Legado de um Pioneiro,NaN,Documentary
146141,Dankyavar Danka,NaN,Comedy
146142,6 Gunn,116.0,None
